# Database | Mini Project 2

### Imports

In [27]:
# utillities
import pandas as pd
import json
from timeit import timeit
from pprint import pprint

# database drivers
from neo4j import GraphDatabase
from pymongo import MongoClient

## Neo4j

In [ ]:
data = pd.read_csv("C:/Users/Hupra/Downloads/data (2).csv")

r = []

for idx, m in data.iterrows():
    d = m.to_dict()
    d["Genre"] = list(map(lambda s: s.strip(), m["Genre"].split(",")))
    d["Actors"] = list(map(lambda s: s.strip(), m["Actors"].split(",")))
    d["Revenue"] = max(0., d["Revenue"])
    d["Metascore"] = max(0., d["Metascore"])
    r.append(d)
    
with open(r'C:\Users\Hupra\Downloads\data.json', 'w') as f:
    json.dump(r, f)

In [ ]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "1234"))

def func1():
    with driver.session() as session:
        m ='''MATCH(y:Year)-[r:RELEASED]-(m:Movie)-[a:ACTED]-(p:Person)
        WHERE y.year = "2006"
        return p.name'''
        r = session.run(m)

def func2():
    with driver.session() as session:
        m ='''MATCH (d:Person)-[:DIRECTED]-(:Movie)-[:ACTED]-(a:Person) 
        WHERE d.name = "David Yates"
        RETURN count(distinct a)'''
        r = session.run(m)

def func3():
    with driver.session() as session:
        m ='''MATCH (:Person {name:"Christian Bale"})-[:ACTED]-(:Movie)-[:GENRE]-(g:Genre) 
        RETURN count(g), g.name'''
        r = session.run(m)

print("func1", timeit(func1, number=5000))
print("func2", timeit(func2, number=5000))
print("func3", timeit(func3, number=5000))

driver.close()

## Mongo DB

In [5]:
mongo_client = MongoClient("mongodb://localhost:27017/")
mongo_db = mongo_client["mini_project"]
mongo_col = mongo_db["main"]
print(mongo_client.list_database_names())

['admin', 'config', 'init', 'local', 'mini_project', 'test', 'twitter']


* **Get name of persons who acted in a movie in 2006**  
* **Get amount of persons that acted in a movie directed by David Yates**  
* **Get genres Christian Bale appeared in**  

In [32]:
response = mongo_col.find({ 'Year':2006 }, { 'Actors':1, '_id':0 })

actors = set()

for document in response:
        actors.update(document['Actors'])

pprint(len(actors))

161


In [ ]:
response = mongo_col.find({ 'Director':'David Yates' }, { 'Actors':1, '_id':0 })

for document in response:
        print(document)

In [ ]:
def mongo_query_1():
    actors = set()
    response = mongo_col.find({ 'Year':2006 }, { 'Actors':1, '_id':0 })

    for document in response:
            actors.update(document['Actors'])

    return actors


def mongo_query_2():
    pass


def mongo_query_3():
    pass


print("mongo query 1", timeit(mongo_query_1, number=5000))
print("mongo query 2", timeit(mongo_query_2, number=5000))
print("mongo query 3", timeit(mongo_query_3, number=5000))